* Load PMI matrix
* Load dictionary
* Calculate scores
* Create plots

In [14]:
import numpy as np
def load_pmi(path):
    pmi = np.load(path)
    return pmi

In [15]:
import pickle
def load_dictionary(path):
    fname = open(path, 'rb')
    data = pickle.load(fname)
    return data

In [20]:
def calc_score_v2(pmi, d, sentence_a, sentence_b):
    s = 0
    pairs = 0
    for word_1 in sentence_a.split():
        for word_2 in sentence_b.split():
            x = d.token2id.get(word_1,-1)
            y = d.token2id.get(word_2,-1)
            if x == -1 or y == -1:
                continue
            s += pmi[x][y]
            pairs += 1
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.true_divide(s, pairs)
    return c

In [18]:
def get_all_scores(pmi, d, all_sentences, k):
    successive = []
    for i in range(len(all_sentences) - k):
        successive.append(calc_score_v2(pmi, d, all_sentences[i], all_sentences[i+k]))
    return successive

In [5]:
def get_all_sentences(data):
    return [line for line in data.split('\n') if len(line)>0]

In [6]:
def read_txt(fname):
    with open(fname, 'r') as f:
        all = f.read()
    return all

In [35]:
import os
import pandas as pd

In [12]:
def iterator(path):
    for fx in os.listdir(path):
        if fx.endswith('.pkl'):
            d = load_dictionary(path+fx)
            
        if fx.endswith('.npy'):
            pmi = load_pmi(path+fx)
            
        if fx.endswith('.txt'):
            data = read_txt(path+fx)
            sentences = get_all_sentences(data)
            
    return pmi, d, sentences

In [40]:
def generator(path, k):
    pmi, d, sentences = iterator(path)
    parent_dir = os.path.basename(os.path.dirname(path))
    name = parent_dir + '_lexical.npy'
    
    lexical = get_all_scores(pmi, d, sentences, k)
    np.save(path + name, lexical)
    
    name = name = parent_dir + '_lexical'
    df = pd.DataFrame(lexical)
    df.to_csv(path + name+'.csv')
    print("Created {}".format(name))

In [45]:
def process_v2(fname):
    all_data = get_data(fname)
    all_data = unidecode.unidecode(all_data)
    sentences = make_sentences(all_data)
    clean_sentences = []
    removed_sentences = []
    for i, sentence in enumerate(sentences):
        t = remove_punc_clean(sentence)
        if len(t) > 0:
            clean_sentences.append(t)
        else:
            removed_sentences.append(i)

    # write_to_file_lexical(clean_sentences, fname)
    print('Done processing', fname)
    return removed_sentences

In [41]:
path = '../final/lexical results/a christmas carol/'
k = 1
generator(path, k)

Created a christmas carol_lexical


In [26]:
path = '../final/lexical results/a tale of 2 cities/'
k = 1
generator(path, k)

Created a tale of 2 cities_lexical.npy


In [27]:
path = '../final/lexical results/great gatsby/'
k = 1
generator(path, k)

Created great gatsby_lexical.npy


In [42]:
path = '../final/lexical results/metamorphosis/'
k = 1
generator(path, k)

Created metamorphosis_lexical


In [29]:
path = '../final/lexical results/mysterious affair/'
k = 1
generator(path, k)

Created mysterious affair_lexical.npy


In [30]:
path = '../final/lexical results/pride and prejudice/'
k = 1
generator(path, k)

Created pride and prejudice_lexical.npy


In [43]:
path = '../final/lexical results/the prophet/'
k = 1
generator(path, k)

Created the prophet_lexical


In [44]:
path = '../final/lexical results/heart of darkness/'
k = 1
generator(path, k)

Created heart of darkness_lexical


In [ ]:
path = '../final/lexical results/a christmas carol/'
for fx in os.listdir(path):
    if fx.endswith('.txt'):
        data = read_txt(path+fx)
        indices = process_v2()
        sentences = get_all_sentences(data)
            

In [46]:
path = '../final/lexical results/a christmas carol/'
pmi_carol, d_carol, sentences_carol = iterator(path)

In [91]:
all_embed = []
for sentence in sentences_carol:
    sentence_embed = []
    for word in sentence.split():
        word_index = d_carol.token2id.get(word, -1)
        sentence_embed.append(pmi_carol[word_index])
        
#     print(len(sentence_embed))
#     print(sentence_embed)
    mat_ = np.asarray(sentence_embed)
#     print(mat_)
#     mean_ = np.mean(mat_, axis = 0)
#     print(mean_.shape)
    all_embed.append(np.mean(mat_, axis = 0 ))
#     break
    
        

In [102]:
f = np.asarray(all_embed)

MemoryError: Unable to allocate 48.6 MiB for an array with shape (1887, 3379) and data type float64

In [90]:
f

array([[0.54332534, 0.54332534, 0.54332534, ..., 0.13278206, 0.22103373,
        0.22103373]])

In [67]:
f

array([[0.54332534, 0.54332534, 0.54332534, ..., 0.13278206, 0.22103373,
        0.22103373],
       [0.72336909, 0.72336909, 0.72336909, ..., 0.21725327, 0.29427851,
        0.29427851],
       [0.89235459, 0.89235459, 0.89235459, ..., 0.31240328, 0.36302461,
        0.36302461],
       ...,
       [0.60295682, 0.60295682, 0.60295682, ..., 0.34615045, 0.24529281,
        0.27651299],
       [0.50169442, 0.50169442, 0.50169442, ..., 0.37951893, 0.20409759,
        0.32273428],
       [0.49132634, 0.49132634, 0.49132634, ..., 0.34309096, 0.19987968,
        0.19987968]])

In [57]:
from sklearn.metrics.pairwise import cosine_similarity

In [54]:
import seaborn as sns
import matplotlib.pyplot as plt

In [70]:
sns.heatmap(cosine_similarity(f,f), square = True)

MemoryError: Unable to allocate 48.6 MiB for an array with shape (1887, 3379) and data type float64